## 情報をとってみる

In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://global.sbisec.co.jp/invest/us/stock/NVDA?resource=price&searchType=include'

res = requests.get(url)

soup = BeautifulSoup(res.text, 'html.parser')

print(soup)

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!-- favicon -->
<link href="https://sbisec.akamaized.net/sbisec/images/common/favicon/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="https://sbisec.akamaized.net/sbisec/images/common/favicon/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="https://sbisec.akamaized.net/sbisec/images/common/favicon/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="https://sbisec.akamaized.net/sbisec/images/common/favicon/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="https://sbisec.akamaized.net/sbisec/images/common/favicon/apple-touch-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="https://sbisec.akamaized.net/sbisec/images/common/favicon/apple-touch-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="https://sbisec.akamaized.net/sb

## selenium使ってみる

In [1]:
import os
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [4]:
from key import user_name, password

ACCOUNT = user_name
PASSWORD = password
# url = 'https://global.sbisec.co.jp/home'
url = 'https://www.sbisec.co.jp/ETGate'

def connect_sbi(ACCOUNT, PASSWORD, url):
    options = Options()
    
    driver = webdriver.Firefox(options=options)
    driver.get(url)

    # ログイン処理
    driver.find_element(By.NAME, 'user_id').send_keys(ACCOUNT)
    driver.find_element(By.NAME, 'user_password').send_keys(PASSWORD)
    driver.find_element(By.NAME, 'ACT_login').click()
    
    return driver

print(connect_sbi(user_name, password, url))

<selenium.webdriver.firefox.webdriver.WebDriver (session="0282f8e9-7e3b-4808-9053-4ba6bd5a76b3")>


In [4]:
import requests

res = requests.post("http://127.0.0.1:8080", data="aaaaaaaaaa")
print(res)
print(res.text)

<Response [200]>
aaaaaaaaaa
